### Goals
- Determine where to save our files
- Decide what subset of movies to retrieve (based on years)
- Develop code to make API calls based on our existing IMDB ID's with inner loop
- Organize output by year into separate .json files using outer loop

<b>
Step 1: [ ] Before the Loops<br>
Step 2: [ ] Create an outer loop for each year<br>
Step 3: [ ] Create an inner loop to make API calls for each id<br>
Step 4: [ ] After the inner loop
    </b>

#### imports

In [1]:
#imports
import pandas as pd
import numpy as np


import os, time, json
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/darlenephan/.secret/tmdb_api.json') as f:
    login=json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
tmdb.API_KEY = login['api-key']

In [4]:
#Designate folder to save your information
FOLDER = 'Data/'
#if folder does exist it will be created
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.DS_Store',
 '.tmdb_api_results_2001.json',
 'tmdb_api_results_2000.json',
 'final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'ratings.csv.gz',
 'title_basics.csv.gz',
 '.tmdb_api_results_2000.json',
 'final_tmdb_data_2001.csv.gz',
 'title_akas.csv.gz']

#### Functions

In [5]:
#get movie ratings
def get_movie_with_rating(movie_id):
    movie = tmdb.Movies(movie_id)
# save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
# Loop through countries in releases
    for c in releases['countries']:
    # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
            info['certification'] = c['certification']
    return info

In [6]:
#write json

def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

Testing functions

In [16]:
get_movie_with_rating('tt0848228')

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 205.168,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [17]:
get_movie_with_rating('tt0332280')

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 56.454,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

#### Read in data and define lists

In [7]:
#Define the years to retrieve
years_to_get = [2000, 2001]

In [8]:
#Define errors list
errors = [ ]

In [9]:
#read in data
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002,NaN,126,Drama


#### API Loop

In [10]:
#set up progress bar for when we iterate through each year in years_to_get
for year in tqdm_notebook(years_to_get, desc='years', position=0):
    
    #defining the json file to store results for year
    JSON_FILE = f"{FOLDER}.tmdb_api_results_{year}.json"
    
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it does not exist - create it
    if file_exists == False:
    #save an empty dictionary with 'imdb_id' to new json file
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb' : 0}], f)
    
    #Defining a new df for each year
    df = basics.loc[basics['startYear'] == year].copy() #year filter same as basics[basics['startYear']]
    #saving movie id from df to list
    movie_ids = df['tconst'].copy()
    
    
    ## Check and remove any previously downloaded movie ids
    #load existing data from json into df 
    previous_df = pd.read_json(JSON_FILE)
    
    #check for and filter out movie ids that already exist in json_file
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb'])]
    
    #-----------Starting inner loop------------#
    
    #get index and move id from list
    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {year}',
                                  position=1, leave=True):
        try:
            #retireve then data for the movie id
            temp = get_movie_with_rating(movie_id)
            #append/extend results to existing file using premade function
            write_json(temp, JSON_FILE)
            
            #short 20 ms sleep to prevent overwheliming server
            time.sleep(.2)
            
        except Exception as e:
            errors.append([movie_id, e])
    
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{year}.csv.gz", compression='gzip', index=False)
print(f"- Total errors: {len(errors)}")
    

years:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1412 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1530 [00:00<?, ?it/s]

- Total errors: 463


#### 2001 Database

In [31]:
df2001 = pd.read_csv('Data/final_tmdb_data_2001.csv.gz')
df2001.head()

,imdb,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,/ab5yL8zgRotrICzGbEl10z24N71.jpg,NaN,48000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 14, 'nam...",NaN,11232.0,tt0035423,en,...,76019048.0,118.0,"[{'english_name': 'Italian', 'iso_639_1': 'it'...",Released,If they lived in the same century they'd be pe...,Kate & Leopold,0.0,6.3,1141.0,PG-13
2,NaN,0.0,NaN,NaN,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",NaN,151007.0,tt0114447,en,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.0,3.0,NaN
3,NaN,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,NaN,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",NaN,10696.0,tt0118589,en,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.6,117.0,PG-13
4,NaN,0.0,/mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg,NaN,1000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",NaN,17140.0,tt0118652,en,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,His search for peace of mind... will leave his...,The Attic Expeditions,0.0,5.1,28.0,R


#### 2000 database 
Df turned up blank so re-running for only 2000

In [21]:
df2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')
df2000

,imdb_id
0,0


In [27]:
year2000 = [2000]

In [28]:
#set up progress bar for when we iterate through each year in years_to_get
for year in tqdm_notebook(year2000, desc='years', position=0):
    
    #defining the json file to store results for year
    JSON_FILE = f"{FOLDER}.tmdb_api_results_{year}.json"
    
    #check if file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it does not exist - create it
    if file_exists == False:
    #save an empty dictionary with 'imdb_id' to new json file
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb' : 0}], f)
    
    #Defining a new df for each year
    df = basics.loc[basics['startYear'] == year].copy() #year filter same as basics[basics['startYear']]
    #saving movie id from df to list
    movie_ids = df['tconst'].copy()
    
    
    ## Check and remove any previously downloaded movie ids
    #load existing data from json into df 
    previous_df = pd.read_json(JSON_FILE)
    
    #check for and filter out movie ids that already exist in json_file
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb'])]
    
    #-----------Starting inner loop------------#
    
    #get index and move id from list
    for movie_id in tqdm_notebook(movie_ids_to_get, desc=f'Movies from {year}',
                                  position=1, leave=True):
        try:
            #retireve then data for the movie id
            temp = get_movie_with_rating(movie_id)
            #append/extend results to existing file using premade function
            write_json(temp, JSON_FILE)
            
            #short 20 ms sleep to prevent overwheliming server
            time.sleep(.2)
            
        except Exception as e:
            errors.append([movie_id, e])
    
final_year_df = pd.read_json(JSON_FILE)
final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{year}.csv.gz", compression='gzip', index=False)
print(f"- Total errors: {len(errors)}")
    

years:   0%|          | 0/1 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1412 [00:00<?, ?it/s]

- Total errors: 692


In [30]:
df2000 = pd.read_csv('Data/final_tmdb_data_2000.csv.gz')
df2000.head()

,imdb,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,tt0113026,en,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
2,NaN,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,tt0113092,en,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,NaN
3,NaN,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,tt0116391,hi,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,0.000,0.0,NaN
4,NaN,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,tt0118694,cn,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.111,1978.0,PG


#### Exploratory Data Analysis

In [40]:
#checking shape of 2000 df
df2000.shape

(2380, 27)

In [41]:
#checking shape of 2001 df
df2001.shape

(1284, 27)

In [43]:
#concat df together on 0 axis
df = pd.concat([df2000, df2001], axis=0)
df.head()

,imdb,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0.0,/vMFs7nw6P0bIV1jDsQpxAieAVnH.jpg,NaN,10000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,62127.0,tt0113026,en,...,0.0,86.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Try to remember the first time magic happened,The Fantasticks,0.0,5.500,22.0,NaN
2,NaN,0.0,NaN,NaN,0.0,"[{'id': 878, 'name': 'Science Fiction'}]",NaN,110977.0,tt0113092,en,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,The ultimate showdown on a forbidden planet.,For the Cause,0.0,5.100,8.0,NaN
3,NaN,0.0,NaN,NaN,0.0,"[{'id': 18, 'name': 'Drama'}, {'id': 28, 'name...",NaN,442869.0,tt0116391,hi,...,0.0,152.0,"[{'english_name': 'Hindi', 'iso_639_1': 'hi', ...",Released,NaN,Gang,0.0,0.000,0.0,NaN
4,NaN,0.0,/n4GJFGzsc7NinI1VeGDXIcQjtU2.jpg,NaN,150000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",http://www.wkw-inthemoodforlove.com/,843.0,tt0118694,cn,...,12854953.0,99.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,"Feel the heat, keep the feeling burning, let t...",In the Mood for Love,0.0,8.111,1978.0,PG


In [44]:
#confirm concat performed properly
df.shape

(3664, 27)

In [45]:
#check df columns
df.columns

Index(['imdb', 'adult', 'backdrop_path', 'belongs_to_collection', 'budget',
       'genres', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'certification'],
      dtype='object')

In [46]:
#confirming release dates have 2000 and 2001 years. 
df['release_date']

0              NaN
1       2000-09-22
2       2000-11-15
3       2000-04-14
4       2000-09-29
           ...    
1279    2001-01-01
1280    2001-01-01
1281    2001-10-10
1282    2001-01-01
1283    2001-07-12
Name: release_date, Length: 3664, dtype: object

In [47]:
#checking for duplicates
df.duplicated().sum()

1173

In [49]:
#drop duplicates
df.drop_duplicates(inplace=True)

In [50]:
#confirm duplicates have been dropped
df.duplicated().sum()

0

In [51]:
#checking df info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2491 entries, 0 to 1283
Data columns (total 27 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   imdb                   1 non-null      float64
 1   adult                  2490 non-null   float64
 2   backdrop_path          1304 non-null   object 
 3   belongs_to_collection  197 non-null    object 
 4   budget                 2490 non-null   float64
 5   genres                 2490 non-null   object 
 6   homepage               175 non-null    object 
 7   id                     2490 non-null   float64
 8   imdb_id                2490 non-null   object 
 9   original_language      2490 non-null   object 
 10  original_title         2490 non-null   object 
 11  overview               2440 non-null   object 
 12  popularity             2490 non-null   float64
 13  poster_path            2202 non-null   object 
 14  production_companies   2490 non-null   object 
 15  prod

In [52]:
#check missing values
df.isna().sum()

imdb                     2490
adult                       1
backdrop_path            1187
belongs_to_collection    2294
budget                      1
genres                      1
homepage                 2316
id                          1
imdb_id                     1
original_language           1
original_title              1
overview                   51
popularity                  1
poster_path               289
production_companies        1
production_countries        1
release_date               19
revenue                     1
runtime                     1
spoken_languages            1
status                      1
tagline                  1448
title                       1
video                       1
vote_average                1
vote_count                  1
certification            1698
dtype: int64

In [76]:
#Saving new df. Dropped duplicates but still has missing values
df.to_csv('Data/tmdb_results_combined.csv.gz', compression='gzip', index=False)

In [54]:
#check statistical data
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
imdb,1.0,0.000000e+00,NaN,0.0,0.000,0.0000,0.000,0.000000e+00
adult,2490.0,2.409639e-03,4.903874e-02,0.0,0.000,0.0000,0.000,1.000000e+00
budget,2490.0,5.340094e+06,1.724310e+07,0.0,0.000,0.0000,0.000,1.400000e+08
id,2490.0,1.712216e+05,2.063764e+05,16.0,26556.000,77953.5000,268731.500,1.006938e+06
popularity,2490.0,5.672033e+00,1.396921e+01,0.6,0.745,1.8285,5.069,2.472490e+02
revenue,2490.0,1.123460e+07,5.314176e+07,0.0,0.000,0.0000,0.000,9.764756e+08
runtime,2490.0,9.234618e+01,3.038409e+01,0.0,87.000,94.0000,104.000,3.450000e+02
video,2490.0,1.606426e-03,4.005608e-02,0.0,0.000,0.0000,0.000,1.000000e+00
vote_average,2490.0,4.552610e+00,2.484691e+00,0.0,3.300,5.3000,6.300,1.000000e+01
vote_count,2490.0,2.624743e+02,1.244584e+03,0.0,1.000,7.0000,41.000,2.324100e+04


In [58]:
#budget greater than 0
df[df['budget'] > 0].shape

(535, 27)

In [67]:
#revenue greater than 0
df[df['revenue'] > 0].shape

(436, 27)

In [69]:
#budget and revenue greater than 0
df[(df['budget'] > 0) | (df['revenue'] > 0)].shape

(622, 27)

In [70]:
#certification categories total count
df['certification'].value_counts()

R          450
PG-13      181
NR          66
PG          64
G           24
NC-17        6
Unrated      1
-            1
Name: certification, dtype: int64

In [75]:
#average revenue and budget per certification category
mean_cat = ['revenue', 'budget']
df.groupby('certification')[mean_cat].mean()

,revenue,budget
certification,,
-,0.000000e+00,0.000000e+00
G,7.216332e+07,2.383333e+07
NC-17,0.000000e+00,0.000000e+00
NR,2.289235e+06,1.622729e+06
PG,6.143535e+07,2.459766e+07
PG-13,7.160834e+07,3.123154e+07
R,1.689589e+07,1.004334e+07
Unrated,0.000000e+00,0.000000e+00
